# Tiền xử lý dữ liệu

## 1. Tổng thể

**Cách làm**

    Nhóm sẽ tiền xử lý các file dữ liệu sau:
        + actors.csv
        + genres.csv
        + tags.csv
        + countries.csv
        + directors.csv
        + userrated_movies.csv

## 2. Bài làm

### Import

In [1]:
import numpy as np
import pandas as pd

### Hàm đọc dữ liệu

In [2]:
def read_data(path, dropped_cols=[], index_col=None, old_names=None, new_names=None):
    '''
    Read data from csv file, drop columns and rename columns
    
    Parameters
    ----------
    path: string
        The string of file .csv
    dropped_cols: list
        List of columns to delete
    index_col: int
        The index of column to set index
    old_names, new_names: list of old names and new names
        To rename from old names to new names
        
    Returns
    ----------
    data_df: DataFrame
        The DataFrame after dropping columns not use
    '''
    data_df = pd.read_csv(path, index_col=index_col) # read data from .csv file
    data_df.drop(dropped_cols, axis=1, inplace=True) # drop columns in dropped_cols
    
    if new_names != None:
        data_df.rename(columns=dict(zip(old_names, new_names)), inplace=True) # rename columns
    return data_df

In [3]:
def compute_mean_mode(data_df):
    '''
    Computes means for numeric input variables and modes for non-numeric ones.
    
    Parameters
    ----------
    train_input_df : data frame
        The data frame containing training inputs.
    
    Returns
    -------
    mean_mode_dict : dictionary, len = # input variables (# columns) of train_input_df
        mean_mode_dict[<column_name>] = mean/mode of this column.
    '''
     
    res = dict()
    
    total_columns = set(data_df.columns)
    numeric_columns = set(data_df._get_numeric_data().columns) # get numeric columns
        
    # res.update({column: data_df[column].mode().iloc[0] for column in total_columns})
    res.update({column: data_df[column].mean() for column in numeric_columns}) # fill mean if numeric column
    res.update({column: data_df[column].mode().iloc[0] for column in total_columns - numeric_columns}) # fill mode if not numeric column
    
    return res


def fill_missing_values(input_df, mean_mode_dict):
    '''
    Fills missing values for ALL columns of `input_df` using `mean_mode_dict`.
    
    Parameters
    ----------
    input_df : data frame
        The data frame containing inputs.
    mean_mode_dict : dictionary
        mean_mode_dict[<column_name>] = mean/mode of this column (estimated from the training set).
    
    Returns
    -------
    filled_input_df : data frame
        The data frame containing inputs after filling missing values.
    '''        
    return input_df.fillna(mean_mode_dict, inplace = True)

## Đọc và chuẩn hóa dữ liệu

### 1. Chuẩn hóa dữ liệu bảng actors và tags

In [4]:
def normalize_data(data_df, old_column_name, new_column_names, idx_col):
    '''
    Normalize DataFrame actors and tags
    
    Parameters
    ----------
    data_df: DataFrame
        DataFrame to normalize
    old_column_name: string
        Name of column to get max, min, avg, std, count
    new_column_names: List of string
        Name of columns max, min, avg, std, count
    idx_col: string
        Name of index column
    
    Returns
    ----------
    res: DataFrame
        New DataFrame with min, max, avg, std, count columns
    '''
    grp = data_df.groupby([idx_col])
    grpmax = grp.max()
    grpmax.rename(columns = {old_column_name:new_column_names[0]}, inplace=True)
    grpmin = grp.min()
    grpmin.rename(columns = {old_column_name:new_column_names[1]}, inplace=True)
    grpavg = grp.mean()
    grpavg.rename(columns = {old_column_name:new_column_names[2]}, inplace=True)
    grpstd = grp.std()
    grpstd.rename(columns = {old_column_name:new_column_names[3]}, inplace=True)
    grpstd.fillna(0, inplace = True)
    grpcnt = grp.count()
    grpcnt.rename(columns = {old_column_name:new_column_names[4]}, inplace=True)

    res = pd.concat([grpmax, grpmin, grpavg, grpstd, grpcnt], axis = 1)
    res.reset_index(inplace = True)
    return res

In [5]:
movie_actors_df = read_data('actors.csv', old_names=['ranking'], new_names=['act_ranking'])
movie_actors_df.head()

,movieID,actorID,actorName,act_ranking
0,1,annie_potts,Annie Potts,10
1,1,bill_farmer,Bill Farmer,20
2,1,don_rickles,Don Rickles,3
3,1,erik_von_detten,Erik von Detten,13
4,1,greg-berg,Greg Berg,17


In [6]:
movie_actors_df = read_data('actors.csv', dropped_cols=['actorID', 'actorName'], old_names=['ranking'], new_names=['act_ranking'])
movie_actors_df.head()

,movieID,act_ranking
0,1,10
1,1,20
2,1,3
3,1,13
4,1,17


In [7]:
new_actor_df = normalize_data(movie_actors_df, 'act_ranking', ['act_max', 'act_min', 'act_avg', 'act_std', 'act_cnt'], 'movieID')
new_actor_df.info()
new_actor_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10174 entries, 0 to 10173
Data columns (total 6 columns):
movieID    10174 non-null int64
act_max    10174 non-null int64
act_min    10174 non-null int64
act_avg    10174 non-null float64
act_std    10174 non-null float64
act_cnt    10174 non-null int64
dtypes: float64(2), int64(4)
memory usage: 477.0 KB


,movieID,act_max,act_min,act_avg,act_std,act_cnt
0,1,24,1,12.5,7.071068,24
1,2,18,1,9.5,5.338539,18
2,3,16,1,8.5,4.760952,16
3,4,20,1,10.5,5.916080,20
4,5,26,1,13.5,7.648529,26


In [8]:
tags_df = read_data('tags.csv', dropped_cols=['tagID']) #, old_names=['ranking'], new_names=['act_ranking'])

new_tag_df = normalize_data(tags_df, 'tagWeight', ['tag_max', 'tag_min', 'tag_avg', 'tag_std', 'tag_cnt'], 'movieID')
new_tag_df.info()
new_tag_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7155 entries, 0 to 7154
Data columns (total 6 columns):
movieID    7155 non-null int64
tag_max    7155 non-null int64
tag_min    7155 non-null int64
tag_avg    7155 non-null float64
tag_std    7155 non-null float64
tag_cnt    7155 non-null int64
dtypes: float64(2), int64(4)
memory usage: 335.5 KB


,movieID,tag_max,tag_min,tag_avg,tag_std,tag_cnt
0,1,17,1,2.186047,3.017850,43
1,2,8,1,1.833333,1.723539,18
2,3,2,1,1.166667,0.408248,6
3,5,2,1,1.500000,0.534522,8
4,6,2,1,1.125000,0.337832,24


### 2. Chuẩn hóa thành one_hot bảng dữ liệu genres

In [9]:
def super_dummy(data_df, _get_dummy_col, element_name, id_col):
    '''
    Get 'super_dummy' by _get_dummy_col column with element_name value
    
    Parameters
    ----------
    data_df: DataFrame
        Input DataFrame
    _get_dummy_col: string
        column to get dummy
    element_name: string
        _get_dummy_col column has 'element name' values
    id_col: string
        column of ID.
    
    Returns
    ----------
    aniDf: DataFrame
        DataFrame after get 'super_dummy' by _get_dummy_col column with element_name value
    '''
     
    aniID = data_df[data_df[_get_dummy_col] == element_name][id_col]
    movieID = data_df[id_col].unique() #list of movieID
    nmovies = len(movieID)

    aniKey = [np.where(movieID == id)[0][0] for id in aniID]

    aniArr = np.asarray(np.zeros((nmovies, 1))) # new array
    aniArr[aniKey] = 1    # array at aniKey = 1
    
    aniDf = pd.DataFrame(aniArr, columns = [element_name])
    return aniDf


def super_dummies(data_df, _get_dummies_col, id_col):
    '''
    Get 'super_dummies' by _get_dummies_col column
    
    Parameters
    ----------
    data_df: DataFrame
        Input DataFrame
    _get_dummies_col: string
        Name of the column to get super_dummies
    id_col: 
        column of ID
    
    Returns
    ----------
    res: DataFrame
        DataFrame after get 'super_dummies' by _get_dummies_col
    '''
    
    data_unique_df = data_df[id_col].unique()
    
    res = pd.DataFrame(data_unique_df, columns = [id_col])
    genres = data_df[_get_dummies_col].unique()
    
    # get super_dummy with each value in column _get_dummies_col
    for g in genres:
        sg = super_dummy(data_df, _get_dummies_col, g, id_col)
        res = pd.concat([res, sg], axis=1) # concatenate columns to original table
    return res

In [10]:
movie_genres_df = read_data('genres.csv')
movie_genres_df.head()

,movieID,genre
0,1,Adventure
1,1,Animation
2,1,Children
3,1,Comedy
4,1,Fantasy


In [11]:
new_genres_df = super_dummies(movie_genres_df, 'genre', 'movieID')
new_genres_df.info()
new_genres_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10197 entries, 0 to 10196
Data columns (total 21 columns):
movieID        10197 non-null int64
Adventure      10197 non-null float64
Animation      10197 non-null float64
Children       10197 non-null float64
Comedy         10197 non-null float64
Fantasy        10197 non-null float64
Romance        10197 non-null float64
Drama          10197 non-null float64
Action         10197 non-null float64
Crime          10197 non-null float64
Thriller       10197 non-null float64
Horror         10197 non-null float64
Mystery        10197 non-null float64
Sci-Fi         10197 non-null float64
IMAX           10197 non-null float64
Documentary    10197 non-null float64
War            10197 non-null float64
Musical        10197 non-null float64
Film-Noir      10197 non-null float64
Western        10197 non-null float64
Short          10197 non-null float64
dtypes: float64(20), int64(1)
memory usage: 1.6 MB


,movieID,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,...,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Film-Noir,Western,Short
0,1,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 3. Lấy trung bình của user rating cho từng bộ phim (theo movieID)

In [12]:
user_ratedmovies_df = read_data('user_ratedmovies.csv')
user_ratedmovies_df.head()

,userID,movieID,rating,date_day,date_month,date_year,date_hour,date_minute,date_second
0,75,3,1.0,29,10,2006,23,17,16
1,75,32,4.5,29,10,2006,23,23,44
2,75,110,4.0,29,10,2006,23,30,8
3,75,160,2.0,29,10,2006,23,16,52
4,75,163,4.0,29,10,2006,23,29,30


In [13]:
user_ratedmovies_df = read_data('user_ratedmovies.csv', ['userID', 'date_day', 'date_month', 'date_year', 'date_hour', 'date_minute', 'date_second'])
user_ratedmovies_df.head()

,movieID,rating
0,3,1.0
1,32,4.5
2,110,4.0
3,160,2.0
4,163,4.0


In [14]:
new_userrated_df = user_ratedmovies_df.groupby('movieID').mean()
new_userrated_df.reset_index(inplace=True)
new_userrated_df.head()

,movieID,rating
0,1,3.735154
1,2,2.976471
2,3,2.873016
3,4,2.577778
4,5,2.753333


### 4. Lấy trung bình rating cho director dựa vào các phim do director đó làm

In [15]:
movie_directors_df = read_data('directors.csv')
movie_directors_df.head()

,movieID,directorID,directorName
0,1,john_lasseter,John Lasseter
1,2,joe_johnston,Joe Johnston
2,3,donald_petrie,Donald Petrie
3,4,forest_whitaker,Forest Whitaker
4,5,charles_shyer,Charles Shyer


In [16]:
# Đọc dữ liệu
movie_directors_df = read_data('directors.csv' , ['directorName'])
movie_directors_df.info()
movie_directors_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10155 entries, 0 to 10154
Data columns (total 2 columns):
movieID       10155 non-null int64
directorID    10155 non-null object
dtypes: int64(1), object(1)
memory usage: 158.7+ KB


,movieID,directorID
0,1,john_lasseter
1,2,joe_johnston
2,3,donald_petrie
3,4,forest_whitaker
4,5,charles_shyer


In [17]:
### TEMP CELL

# Lấy trung bình rating cho từng director
data_merge = pd.merge(new_userrated_df, movie_directors_df, on='movieID', how='outer')
rating_director = data_merge.groupby(['directorID']).mean()
rating_director.reset_index(inplace=True)
rating_director.drop(['movieID'], axis=1, inplace=True)


data_merge = pd.merge(movie_directors_df, rating_director, on='directorID', how='outer')
data_merge.drop(['directorID'], axis=1, inplace=True)

# Đổi tên cột
new_directors_df = data_merge.rename(columns={'rating': 'director_rating'})
new_directors_df.info()
new_directors_df.head(10)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10155 entries, 0 to 10154
Data columns (total 2 columns):
movieID            10155 non-null int64
director_rating    10126 non-null float64
dtypes: float64(1), int64(1)
memory usage: 238.0 KB


,movieID,director_rating
0,1,3.438840
1,2355,3.438840
2,3114,3.438840
3,4929,3.438840
4,45517,3.438840
5,2,3.081553
6,2054,3.081553
7,2094,3.081553
8,2501,3.081553
9,4638,3.081553


### 5. Tương tự như như trên, ta sẽ lấy trung bình rating cho countries dựa vào các phim được sản xuất tại country đó

In [18]:
# Đọc dữ liệu
movie_countries_df = read_data('countries.csv')
movie_countries_df.info()
movie_countries_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10197 entries, 0 to 10196
Data columns (total 2 columns):
movieID    10197 non-null int64
country    10189 non-null object
dtypes: int64(1), object(1)
memory usage: 159.4+ KB


,movieID,country
0,1,USA
1,2,USA
2,3,USA
3,4,USA
4,5,USA


In [19]:
# ### TEMP CELL


# # Thêm giá trị thiếu cho các cột giá trị country của bảng Countries
# mean_mode_dict_countries = compute_mean_mode(movie_countries_df)
# movie_countries_df = fill_missing_values(movie_countries_df, mean_mode_dict_countries)

# # Lấy trung bình rating cho từng country
# data_merge = pd.merge(new_userrated_df, movie_countries_df, on='movieID', how='outer')
# rating_country = data_merge.groupby(['country']).mean()
# rating_country.reset_index(inplace=True)
# rating_country.drop(['movieID'], axis=1, inplace=True)


# data_merge = pd.merge(movie_countries_df, rating_country, on='country', how='outer')
# data_merge.drop(['country'], axis=1, inplace=True)

# # Đổi tên cột
# new_countries_df = data_merge.rename(columns={'rating': 'country_rating'})
# new_countries_df.info()
# new_countries_df.head()

### 6. Nối các bảng lại với nhau theo movieID

In [20]:
# Nối các bảng dữ liệu lại với nhau theo movieID
movie_total_df = pd.merge(new_actor_df, new_genres_df, on='movieID', how='inner')
# movie_total_df = movie_total_df.merge(new_userrated_df, on='movieID', how='inner')


movie_total_df = movie_total_df.merge(movie_directors_df, on='movieID', how='inner')
movie_total_df = movie_total_df.merge(movie_countries_df, on='movieID', how='inner')

# Bỏ bớt các cột không giúp ích nhiều cho việc huấn luyện dữ liệu
# movie_total_df.drop(['movieID', 'act_std', 'act_cnt', 'act_min', 'act_avg'], axis=1, inplace=True)
movie_total_df.drop(['act_std', 'act_cnt', 'act_min', 'act_avg'], axis=1, inplace=True)

movie_total_df.info()
movie_total_df.head(20)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10134 entries, 0 to 10133
Data columns (total 24 columns):
movieID        10134 non-null int64
act_max        10134 non-null int64
Adventure      10134 non-null float64
Animation      10134 non-null float64
Children       10134 non-null float64
Comedy         10134 non-null float64
Fantasy        10134 non-null float64
Romance        10134 non-null float64
Drama          10134 non-null float64
Action         10134 non-null float64
Crime          10134 non-null float64
Thriller       10134 non-null float64
Horror         10134 non-null float64
Mystery        10134 non-null float64
Sci-Fi         10134 non-null float64
IMAX           10134 non-null float64
Documentary    10134 non-null float64
War            10134 non-null float64
Musical        10134 non-null float64
Film-Noir      10134 non-null float64
Western        10134 non-null float64
Short          10134 non-null float64
directorID     10134 non-null object
country        10129 n

,movieID,act_max,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,...,Sci-Fi,IMAX,Documentary,War,Musical,Film-Noir,Western,Short,directorID,country
0,1,24,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,john_lasseter,USA
1,2,18,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,joe_johnston,USA
2,3,16,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,donald_petrie,USA
3,4,20,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,forest_whitaker,USA
4,5,26,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,charles_shyer,USA
5,6,47,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,michael_mann,USA
6,7,27,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,billy_wilder,USA
7,8,15,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1052316-peter_hewitt,USA
8,9,72,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,peter_hyams,USA
9,10,20,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,martin_campbell,UK


In [21]:
train_movie_total_df = movie_total_df[: int(0.6 * len(movie_total_df))]
val_movie_total_df = movie_total_df[int(0.6 * len(movie_total_df)) : int(0.8 * len(movie_total_df))]
test_movie_total_df = movie_total_df[int(0.8 * len(movie_total_df)) : len(movie_total_df)]

In [22]:
train_movie_total_df.info()
train_movie_total_df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6080 entries, 0 to 6079
Data columns (total 24 columns):
movieID        6080 non-null int64
act_max        6080 non-null int64
Adventure      6080 non-null float64
Animation      6080 non-null float64
Children       6080 non-null float64
Comedy         6080 non-null float64
Fantasy        6080 non-null float64
Romance        6080 non-null float64
Drama          6080 non-null float64
Action         6080 non-null float64
Crime          6080 non-null float64
Thriller       6080 non-null float64
Horror         6080 non-null float64
Mystery        6080 non-null float64
Sci-Fi         6080 non-null float64
IMAX           6080 non-null float64
Documentary    6080 non-null float64
War            6080 non-null float64
Musical        6080 non-null float64
Film-Noir      6080 non-null float64
Western        6080 non-null float64
Short          6080 non-null float64
directorID     6080 non-null object
country        6077 non-null object
dtypes: flo

,movieID,act_max,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,...,Sci-Fi,IMAX,Documentary,War,Musical,Film-Noir,Western,Short,directorID,country
0,1,24,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,john_lasseter,USA
1,2,18,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,joe_johnston,USA
2,3,16,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,donald_petrie,USA
3,4,20,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,forest_whitaker,USA
4,5,26,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,charles_shyer,USA


In [23]:
# Lấy trung bình rating cho từng director
movie_directors_df_2 = train_movie_total_df[['movieID', 'directorID']]

data_merge = pd.merge(new_userrated_df, movie_directors_df_2, on='movieID', how='outer')
rating_director = data_merge.groupby(['directorID']).mean()
rating_director.reset_index(inplace=True)
rating_director.drop(['movieID'], axis=1, inplace=True)
rating_director = rating_director.rename(columns={'rating': 'director_rating'})


data_merge = pd.merge(movie_directors_df_2, rating_director, on='directorID', how='outer')
data_merge.drop(['directorID'], axis=1, inplace=True)

# Dữ liệu mới
new_directors_df = data_merge 
new_directors_df.info()
new_directors_df.head(10)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6080 entries, 0 to 6079
Data columns (total 2 columns):
movieID            6080 non-null int64
director_rating    6064 non-null float64
dtypes: float64(1), int64(1)
memory usage: 142.5 KB


,movieID,director_rating
0,1,3.381533
1,2355,3.381533
2,3114,3.381533
3,4929,3.381533
4,2,3.038124
5,2054,3.038124
6,2094,3.038124
7,2501,3.038124
8,4638,3.038124
9,3,2.781743


In [24]:
# Lấy trung bình rating cho từng country
movie_countries_df_2 = train_movie_total_df[['movieID', 'country']]
data_merge = pd.merge(new_userrated_df, movie_countries_df_2, on='movieID', how='outer')
rating_country = data_merge.groupby(['country']).mean()
rating_country.reset_index(inplace=True)
rating_country.drop(['movieID'], axis=1, inplace=True)

rating_country = rating_country.rename(columns={'rating':'country_rating'})


data_merge = pd.merge(movie_countries_df_2, rating_country, on='country', how='outer')
data_merge.drop(['country'], axis=1, inplace=True)

# Dữ liệu mới
new_countries_df = data_merge
new_countries_df.info()
new_countries_df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6080 entries, 0 to 6079
Data columns (total 2 columns):
movieID           6080 non-null int64
country_rating    6077 non-null float64
dtypes: float64(1), int64(1)
memory usage: 142.5 KB


,movieID,country_rating
0,1,3.089956
1,2,3.089956
2,3,3.089956
3,4,3.089956
4,5,3.089956


In [25]:
train_movie_total_df_new = train_movie_total_df.drop(['directorID', 'country'], axis=1)

train_movie_total_df_new = train_movie_total_df_new.merge(new_directors_df, on='movieID', how='outer')
train_movie_total_df_new = train_movie_total_df_new.merge(new_countries_df, on='movieID', how='outer')

train_movie_total_df_new = train_movie_total_df_new.merge(new_userrated_df, on='movieID', how='outer')

mean_mode_dict = compute_mean_mode(train_movie_total_df_new)
train_movie_total_df_new = fill_missing_values(train_movie_total_df_new, mean_mode_dict)

train_movie_total_df_new.drop(['movieID'], axis=1, inplace=True)
train_movie_total_df_new.head()

,act_max,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,...,IMAX,Documentary,War,Musical,Film-Noir,Western,Short,director_rating,country_rating,rating
0,24.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.381533,3.089956,3.735154
1,18.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.038124,3.089956,2.976471
2,16.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.781743,3.089956,2.873016
3,20.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.670911,3.089956,2.577778
4,26.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.041643,3.089956,2.753333


### Chuẩn hóa tập validation và tập test.

In [26]:
rating_country.head()
rating_director.head()

,directorID,director_rating
0,1000993-bruce_barry,0.500000
1,1001257-peter_berg,2.719512
2,1006013-keith_gordon,3.530952
3,1006266-paul_gross,3.216667
4,1009699-bill_maher,2.480769


In [27]:
data_merge = pd.merge(val_movie_total_df, rating_director, on='directorID', how='outer')
data_merge = data_merge.merge(rating_country, on='country', how='outer')

data_merge = fill_missing_values(data_merge, mean_mode_dict)

val_movie_total_df_new = data_merge.merge(new_userrated_df, on='movieID', how='inner')
val_movie_total_df_new = val_movie_total_df_new.drop(['movieID', 'directorID', 'country'], axis=1)
val_movie_total_df_new.head()

,act_max,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,...,IMAX,Documentary,War,Musical,Film-Noir,Western,Short,director_rating,country_rating,rating
0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.166349,3.450601,3.166667
1,3.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.166349,3.450601,4.200000
2,11.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.166349,3.450601,3.800000
3,15.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.197156,3.450601,3.742188
4,16.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.166349,3.450601,3.000000


In [28]:
data_merge = pd.merge(test_movie_total_df, rating_director, on='directorID', how='outer')
data_merge = data_merge.merge(rating_country, on='country', how='outer')

data_merge = fill_missing_values(data_merge, mean_mode_dict)

test_movie_total_df_new = data_merge.merge(new_userrated_df, on='movieID', how='inner')
test_movie_total_df_new = test_movie_total_df_new.drop(['movieID', 'directorID', 'country'], axis=1)
test_movie_total_df_new.head()

,act_max,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,...,IMAX,Documentary,War,Musical,Film-Noir,Western,Short,director_rating,country_rating,rating
0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.062500,3.089956,3.464286
1,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.686441,3.089956,3.653846
2,21.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.686441,3.089956,3.000000
3,26.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.162122,3.089956,3.863636
4,17.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3.162122,3.089956,3.465000


### 7. Lưu dữ liệu đã tiền xử lý xuống file .csv

In [29]:
train_movie_total_df_new.to_csv('hec_train.csv')
train_movie_total_df_new.info()

val_movie_total_df_new.to_csv('hec_validation.csv')
val_movie_total_df_new.info()
test_movie_total_df_new.to_csv('hec_test.csv')
test_movie_total_df_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10143 entries, 0 to 10142
Data columns (total 24 columns):
act_max            10143 non-null float64
Adventure          10143 non-null float64
Animation          10143 non-null float64
Children           10143 non-null float64
Comedy             10143 non-null float64
Fantasy            10143 non-null float64
Romance            10143 non-null float64
Drama              10143 non-null float64
Action             10143 non-null float64
Crime              10143 non-null float64
Thriller           10143 non-null float64
Horror             10143 non-null float64
Mystery            10143 non-null float64
Sci-Fi             10143 non-null float64
IMAX               10143 non-null float64
Documentary        10143 non-null float64
War                10143 non-null float64
Musical            10143 non-null float64
Film-Noir          10143 non-null float64
Western            10143 non-null float64
Short              10143 non-null float64
director_